In [2]:
import os
import sys
import glob
from os import write

from tqdm import tqdm
import pandas as pd
import numpy as np
import ast

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sys.path.append(os.path.realpath(os.path.abspath('..')))
from iDrink import iDrinkUtilities
from iDrink.iDrinkUtilities import get_title_measure_name, get_unit, get_cad, get_setting_axis_name

In [3]:
drive = iDrinkUtilities.get_drivepath()

root_iDrink = os.path.join(drive, 'iDrink')
root_val = os.path.join(root_iDrink, "validation_root")
root_stat = os.path.join(root_val, '04_Statistics')
root_omc = os.path.join(root_val, '03_data', 'OMC_new', 'S15133')
root_data = os.path.join(root_val, "03_data")
root_logs = os.path.join(root_val, "05_logs")

csv_val_trials = os.path.join(root_logs, 'validation_trials.csv')
df_val_trials = pd.read_csv(csv_val_trials, sep=';')

csv_settings = os.path.join(root_logs, 'validation_settings.csv')
df_settings = pd.read_csv(csv_settings, sep=';')

csv_calib_error = os.path.join(root_logs, 'calib_errors.csv')
df_calib_error = pd.read_csv(csv_calib_error, sep=';')

csv_murphy = os.path.join(root_stat, '02_categorical', 'murphy_measures.csv')
df_murphy = pd.read_csv(csv_murphy, sep=';')

csv_failed_trials = os.path.join(root_stat, '04_failed_trials', 'failed_trials.csv')
df_failed_trials = pd.read_csv(csv_failed_trials, sep=';')


csv_cad = os.path.join(root_stat, '02_categorical', 'clinically_acceptable_difference.csv')

df_cad = pd.read_csv(csv_cad, sep=',')

list_identifier = sorted(df_val_trials['identifier'].tolist())

ignore_id_p = ['P11', 'P19']
idx_s_singlecam = ['S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026']
idx_s_multicam = ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015', 'S016']
idx_s_multicam_reduced = ['S001', 'S002']
idx_s_reduced = idx_s_multicam_reduced + idx_s_singlecam
idx_s = idx_s_multicam + idx_s_singlecam

# Range of Motion Error
## Time-Series

In [7]:
from scipy import stats

dir_stat_cont = os.path.join(root_stat, '01_continuous')
dir_results = os.path.join(dir_stat_cont, '01_results')
dir_plots = os.path.join(dir_stat_cont, '02_plots')

csv_rom = os.path.join(dir_results, 'omc_mmc_rom.csv')
df_rom = pd.read_csv(csv_rom, sep=';')
df_rom.dropna(axis=1, inplace=True)
dir_boxplots_rom_error = os.path.join(dir_plots, '04_rom', '01_rom_error_boxplots')

df_rom = df_rom[(df_rom['id_s'].isin(idx_s_reduced)) & (~df_rom['id_p'].isin(ignore_id_p))]

# Detect outlier for ROM errors
cols_error = ['hand_vel_rom_error', 'elbow_vel_rom_error', 'trunk_disp_rom_error', 'trunk_ang_rom_error', 'elbow_flex_pos_rom_error', 'shoulder_flex_pos_rom_error', 'shoulder_abduction_pos_rom_error']
df_rom = df_rom[(np.abs(stats.zscore(df_rom[cols_error])) < 3).all(axis=1)]


#metrics = ['hand_vel', 'elbow_vel', 'trunk_disp', 'trunk_ang', 'elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']
metrics = ['hand_vel', 'elbow_vel', 'trunk_disp', 'elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']

df_filtered = df_rom[~df_rom['id_p'].isin(ignore_id_p)].sort_values(by='id_p')
df_filtered = df_filtered[df_filtered['dynamic'] == 'fixed']
df_filtered_rom_errors = df_filtered.groupby(['id_s', 'id_t']).median(numeric_only=True).filter(like='error')

df_for_plot = pd.DataFrame(columns=['id_s', 'id_s_name', 'id_t', 'measure', 'error'])
joint_metrics = ['elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']

for id in df_filtered_rom_errors.index.to_list():
    id_s = id[0]
    id_t = id[1]
    id_s_name = get_setting_axis_name(id_s)
    for metric in metrics:
        error = df_filtered_rom_errors.loc[id, f'{metric}_rom_error']
        df_for_plot = pd.concat([df_for_plot, pd.DataFrame({'id_s': id_s, 'id_s_name': id_s_name, 'id_t': id_t, 'measure': metric, 'error': error}, index=[0])], ignore_index=True)
    

In [10]:
os.makedirs(dir_boxplots_rom_error, exist_ok=True)

for measure in metrics:
    unit = get_unit(measure)
    title_name = get_title_measure_name(measure)
        
    cad = get_cad(df_cad, measure)
    cad_neg = -cad
    
    fig = px.box(df_for_plot[df_for_plot['measure'] == measure].sort_values(by='id_s'), x='id_s_name', y='error', title=f'{title_name} ROM error, cad = {cad}',
                 template='plotly', hover_name='id_t')
    

    
    fig.add_hline(y=cad, line_dash='dash', line_color='red', name='CAD')
    fig.add_hline(y=cad_neg, line_dash='dash', line_color='red', name='CAD')
    
    fig.update_layout(title= f'{title_name} ROM error, cad = {cad} {unit}',
                  xaxis_title=f'Settings',
                  yaxis_title=f'ROM error [{unit}]',
                  )
    
    fig.show()
    
    filepath = os.path.join(dir_boxplots_rom_error, f'{measure}_rom_error_boxplot.html')
    fig.write_html(filepath)
    
    filepath = os.path.join(dir_boxplots_rom_error, f'{measure}_rom_error_boxplot.png')
    fig.write_image(filepath, scale=5)
    
    filepath = os.path.join(dir_boxplots_rom_error, f'{measure}_rom_error_boxplot.svg')
    fig.write_image(filepath)
    

In [219]:
ignore_id_s = ['S003', 'S005', 'S006', 'S007', 'S008']

In [220]:
df_filtered = df_rom[~df_rom['id_p'].isin(ignore_id_p)].sort_values(by='id_p')
df_filtered = df_filtered[~df_filtered['id_s'].isin(ignore_id_s)].sort_values(by='id_p')


df_filtered = df_filtered[df_filtered['dynamic'] == 'fixed']
df_filtered_rom_errors = df_filtered.groupby(['id_s', 'id_t']).mean(numeric_only=True).filter(like='error')

df_for_plot_corr = pd.DataFrame(columns=['id_s', 'id_t', 'measure', 'error'])
joint_metrics = ['elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']

for id in df_filtered_rom_errors.index.to_list():
    id_s = id[0]
    id_t = id[1]
    for metric in metrics:
        error = df_filtered_rom_errors.loc[id, f'{metric}_rom_error']
        df_for_plot_corr = pd.concat([df_for_plot_corr, pd.DataFrame({'id_s': [id_s], 'id_t': [id_t], 'measure': [metric], 'error': [error]})], ignore_index=True)
    

In [221]:
df_checkup = df_for_plot.groupby(['id_s', 'measure']).median(numeric_only=True)

In [222]:
os.makedirs(dir_boxplots_rom_error, exist_ok=True)

for measure in metrics:
    unit = get_unit(measure)
    title_name = get_title_measure_name(measure)
    
    cad = get_cad(df_cad, measure)
    cad_minus = -cad
    
    fig = px.box(df_for_plot_corr[df_for_plot_corr['measure'] == measure], x='id_s', y='error', title=f'{title_name} ROM error, cad = {cad} {unit}',
                 template='plotly')
    

    
    fig.add_hline(y=cad, line_dash='dash', line_color='red', name='CAD')
    fig.add_hline(y=-cad, line_dash='dash', line_color='red', name='CAD')
    
    fig.update_layout(title= f'{title_name} ROM error, cad = {cad} {unit}',
                  xaxis_title=f'Settings',
                  yaxis_title=f'ROM error [{unit}]',
                  )
    
    fig.show()
    
    filepath = os.path.join(dir_boxplots_rom_error, f'{measure}_rom_error_boxplot_corr.html')
    fig.write_html(filepath)
    
    filepath = os.path.join(dir_boxplots_rom_error, f'{measure}_rom_error_boxplot_corr.png')
    fig.write_image(filepath)

## barplots for rmse


In [223]:
dir_stat_cont = os.path.join(root_stat, '01_continuous')
dir_results = os.path.join(dir_stat_cont, '01_results')
dir_plots = os.path.join(dir_stat_cont, '02_plots')

csv_rom_rmse = os.path.join(dir_results, 'omc_mmc_rom_rmse.csv')
df_rom_rmse = pd.read_csv(csv_rom_rmse, sep=';')

metrics = ['hand_vel', 'elbow_vel', 'trunk_disp', 'elbow_flex_pos', 'shoulder_flex_pos',
           'shoulder_abduction_pos']

In [230]:
# Detect outlier for ROM errors
cols_error = ['hand_vel_rom_error', 'elbow_vel_rom_error', 'trunk_disp_rom_error', 'trunk_ang_rom_error', 'elbow_flex_pos_rom_error', 'shoulder_flex_pos_rom_error', 'shoulder_abduction_pos_rom_error']
df_rom = df_rom[(np.abs(stats.zscore(df_rom[cols_error])) < 3).all(axis=1)]

df_rom_rmse.dropna(axis=0, inplace=True)
dir_barplots_rom_rmse_error = os.path.join(dir_plots, '04_rom', '02_rom_rmse_barplots')

"""for metric in metrics:
    mask = np.abs(stats.zscore(df_rom_rmse[df_rom_rmse['metric'] == metric]['rom_rmse'])) < 3
    mask = mask.reindex(df_rom_rmse.index, fill_value=True)
    df_rom_rmse = df_rom_rmse.loc[mask]"""

df_filtered = df_rom_rmse[~df_rom_rmse['id_p'].isin(ignore_id_p)].sort_values(by='id_p')
df_filtered = df_filtered[df_filtered['dynamic'] == 'fixed']
df_filtered = df_filtered.groupby(['id_s', 'metric']).mean(numeric_only=True)

df_for_plot = pd.DataFrame(columns=['id_s', 'metric', 'rom_rmse'])
joint_metrics = ['elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']

for id in df_filtered.index.to_list():
    id_s = id[0]
    metric = id[1]
    rmse = df_filtered.loc[id, 'rom_rmse']
    df_for_plot = pd.concat([df_for_plot, pd.DataFrame({'id_s': [id_s], 'metric': [metric], 'rom_rmse': [rmse]})], ignore_index=True)


os.makedirs(dir_barplots_rom_rmse_error, exist_ok=True)

for metric in metrics:
    cad = get_cad(df_cad, metric)
    unit = get_unit(metric)
    title_name = get_title_measure_name(metric)
    fig = px.bar(df_for_plot[df_for_plot['metric'] == metric], x='id_s', y='rom_rmse', title=f'{title_name} ROM RMSE, cad = {cad}',
                 template='plotly', )
    
    
    
    fig.add_hline(y=cad, line_dash='dash', line_color='red', name='CAD')
    
    fig.show()
    
    fig.update_layout(title= f'{title_name} ROM RMSE, cad = {cad} {unit}',
                  xaxis_title=f'Settings',
                  yaxis_title=f'ROM RMSE [{unit}]',
                  )
    
    filepath = os.path.join(dir_barplots_rom_rmse_error, f'{metric}_rom_rmse_barplot.html')
    fig.write_html(filepath)
    
    filepath = os.path.join(dir_barplots_rom_rmse_error, f'{metric}_rom_rmse_barplot.png')
    fig.write_image(filepath)

In [225]:
df_for_plot.groupby(['id_s', 'metric']).median(numeric_only=True)

rom_rmse
id_s metric                             
S001 elbow_flex_pos             5.688395
     elbow_vel                  1.143711
     hand_vel                 107.659963
     shoulder_abduction_pos     3.297573
     shoulder_flex_pos          5.001351
...                                  ...
S026 hand_vel                1187.781651
     shoulder_abduction_pos     2.690177
     shoulder_flex_pos          2.152301
     trunk_ang                  3.085525
     trunk_disp                25.336321

[182 rows x 1 columns]

In [226]:
df_for_plot.groupby(['id_s', 'metric']).mean(numeric_only=True)

rom_rmse
id_s metric                             
S001 elbow_flex_pos             5.688395
     elbow_vel                  1.143711
     hand_vel                 107.659963
     shoulder_abduction_pos     3.297573
     shoulder_flex_pos          5.001351
...                                  ...
S026 hand_vel                1187.781651
     shoulder_abduction_pos     2.690177
     shoulder_flex_pos          2.152301
     trunk_ang                  3.085525
     trunk_disp                25.336321

[182 rows x 1 columns]

In [231]:
#ignore_id_s = ['S003', 'S005', 'S006', 'S007', 'S008']
ignore_id_s = ['S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015', 'S016']

df_filtered = df_rom_rmse[~df_rom_rmse['id_s'].isin(ignore_id_s)].sort_values(by='id_p')
df_rom_rmse_corr = df_filtered.groupby(['id_s', 'metric']).mean(numeric_only=True)

df_for_plot_corr_rmse = pd.DataFrame(columns=['id_s', 'metric', 'rom_rmse'])
joint_metrics = ['elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']

for id in df_rom_rmse_corr.index.to_list():
    id_s = id[0]
    metric = id[1]
    rmse = df_rom_rmse_corr.loc[id, 'rom_rmse']
    df_for_plot_corr_rmse = pd.concat([df_for_plot_corr_rmse, pd.DataFrame({'id_s': [id_s], 'metric': [metric], 'rom_rmse': [rmse]})], ignore_index=True)

os.makedirs(dir_barplots_rom_rmse_error, exist_ok=True)

for metric in metrics:
    cad = get_cad(df_cad, metric)
    unit = get_unit(metric)
    title_name = get_title_measure_name(metric)
    fig = px.bar(df_for_plot_corr_rmse[df_for_plot_corr_rmse['metric'] == metric], x='id_s', y='rom_rmse', title=f'{title_name} ROM RMSE, cad = {cad}',
                 template='plotly')
    
    fig.add_hline(y=cad, line_dash='dash', line_color='red', name='CAD')
    
    fig.show()
    
    fig.update_layout(title= f'{title_name} ROM RMSE, cad = {cad} {unit}',
                  xaxis_title=f'Settings',
                  yaxis_title=f'ROM RMSE [{unit}]',
                  )
    
    filepath = os.path.join(dir_barplots_rom_rmse_error, f'{metric}_rom_rmse_barplot_corr.html')
    fig.write_html(filepath)
    
    filepath = os.path.join(dir_barplots_rom_rmse_error, f'{metric}_rom_rmse_barplot_corr.png')
    fig.write_image(filepath)

In [228]:
df_for_plot_corr_rmse

,id_s,metric,rom_rmse
0,S001,elbow_flex_pos,8.557819
1,S001,elbow_vel,1.188189
2,S001,hand_vel,130.153673
3,S001,shoulder_abduction_pos,4.054584
4,S001,shoulder_flex_pos,6.693810
...,...,...,...
79,S026,hand_vel,1140.809400
80,S026,shoulder_abduction_pos,2.836088
81,S026,shoulder_flex_pos,3.827721
82,S026,trunk_ang,4.299640


In [229]:
df_for_plot_corr_rmse.groupby(['id_s', 'metric']).mean(numeric_only=True)

rom_rmse
id_s metric                             
S001 elbow_flex_pos             8.557819
     elbow_vel                  1.188189
     hand_vel                 130.153673
     shoulder_abduction_pos     4.054584
     shoulder_flex_pos          6.693810
...                                  ...
S026 hand_vel                1140.809400
     shoulder_abduction_pos     2.836088
     shoulder_flex_pos          3.827721
     trunk_ang                  4.299640
     trunk_disp                38.803100

[84 rows x 1 columns]

# Correlation
